In [4]:
!pip install tensorflow tensorflow_hub


In [3]:
from google.colab import files

# This will prompt you to upload the zip file from your local machine
uploaded = files.upload()


Saving Dataset Brain Tumor.zip to Dataset Brain Tumor (1).zip


In [10]:
import zipfile
import os

# Specify the name of the uploaded zip file
zip_file_name = 'Dataset Brain Tumor.zip'  # Change this if your zip file name is different

# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall('Dataset Brain Tumor')

# Verify the contents of the unzipped directory
os.listdir('Dataset Brain Tumor')

['Dataset Brain Tumor']

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)


# Load training data with augmentation
train_generator = datagen.flow_from_directory(
    '/content/Dataset Brain Tumor/Dataset Brain Tumor/Training Dataset',
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical'
)

# Load testing data without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/Dataset Brain Tumor/Dataset Brain Tumor/Testing Dataset',
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical'
)


Found 5708 images belonging to 4 classes.
Found 1273 images belonging to 4 classes.


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model

# Define data augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Reduced range
    width_shift_range=0.2,  # Reduced range
    height_shift_range=0.2,  # Reduced range
    shear_range=0.2,  # Reduced range
    zoom_range=0.2,  # Reduced range
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data with augmentation
train_generator = datagen.flow_from_directory(
    '/content/Dataset Brain Tumor/Dataset Brain Tumor/Training Dataset',
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical'
)

# Load testing data without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/Dataset Brain Tumor/Dataset Brain Tumor/Testing Dataset',
    target_size=(240, 240),
    batch_size=32,
    class_mode='categorical'
)

# Load the base model
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)  # Added L2 regularization
x = Dropout(0.5)(x)  # Apply dropout
x = Dense(4, activation='softmax')(x)  # Output layer for 4 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=test_generator,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)

# Step 3: Evaluate the best model

# Load the best model based on validation accuracy
best_model = load_model('best_model.h5')

# Evaluate the performance of the best model on the test dataset
evaluation = best_model.evaluate(test_generator)
print(f"Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}")


Found 5708 images belonging to 4 classes.
Found 1273 images belonging to 4 classes.
Epoch 1/30
179/179 [==============================] - ETA: 0s - loss: 1.6980 - accuracy: 0.8367

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


179/179 [==============================] - 217s 844ms/step - loss: 1.6980 - accuracy: 0.8367 - val_loss: 2.6975 - val_accuracy: 0.3181 - lr: 1.0000e-04
Epoch 2/30
179/179 [==============================] - 146s 812ms/step - loss: 1.3018 - accuracy: 0.9445 - val_loss: 2.4951 - val_accuracy: 0.3181 - lr: 1.0000e-04
Epoch 3/30
179/179 [==============================] - 148s 823ms/step - loss: 1.1099 - accuracy: 0.9650 - val_loss: 1.5460 - val_accuracy: 0.7502 - lr: 1.0000e-04
Epoch 4/30
179/179 [==============================] - 146s 811ms/step - loss: 0.9592 - accuracy: 0.9769 - val_loss: 2.6085 - val_accuracy: 0.4619 - lr: 1.0000e-04
Epoch 5/30
179/179 [==============================] - 149s 830ms/step - loss: 0.8246 - accuracy: 0.9807 - val_loss: 4.5175 - val_accuracy: 0.3221 - lr: 1.0000e-04
Epoch 6/30
179/179 [==============================] - 151s 842ms/step - loss: 0.7133 - accuracy: 0.9811 - val_loss: 0.8404 - val_accuracy: 0.9293 - lr: 1.0000e-04
Epoch 7/30
179/179 [=============